# EPL vs ESL Speaks

## 1 Import and Preprocess Data

### 1.1 Import data and delete unwanted datapoints
1. Copy speaker tabs from WUDC2017, EUDC2016, Oxford IV 2016, Cambridge IV 2016.

2. Remove speakers who have not completed all five rounds or are swing speakers from the tab.
    * If speakers have not completed all five rounds, they may have been ill and so other confounding factors likely affected their performance.
    * Swing speakers may not be  taking the tournament seriously. Their on-tab names may also be wrong.
        * Took out Swing A in EUDC2016 for completeness even though 
    * The average speaks also tend to wrong in that they are an average across rounds-including-those-where-said-speaker-got-zero-speaks.

### 1.2 Convert data into desired format
1. Create dummy variable columns for EPL, ESL, EFL and ESL_or_EFL.
    * EFL applies only to WUDC and EUDC.
    * Distinguishing between ESL and EFL speakers may be of interest, so we will keep both categories and add an `ESL_or_EFL` dummy variable.
    * Method: use formula `=IF(C2,1,0)`, for each cell in the `is_esl` column where column C is the speaker's ESL rank and 2 is the speaker's row number.
2. Create datasets:
    * (i) Use average speaks in a tournament per speaker only.
    * 

In [23]:
import pandas as pd
from collections import defaultdict

In [37]:
# Read in tab data (with `is_esl` column)
eudc2016 = pd.read_csv("eudc2016.csv")
wudc2017 = pd.read_csv("wudc2017.csv")
oxiv2016 = pd.read_csv("oxiv2016.csv")
camiv2016 = pd.read_csv("camiv2016.csv")

In [65]:
# Preview one tab
eudc2016.head()

,is_esl,ENL,ESL,Speaker,Team,Team Points,Speaker Points,AVG,#1,#2,#3,#4,#5,#6,#7,#8,#9
0,0,1,NaN,Michael Dunn Goekjian,PEP A,22,762,84.7,87,83,84,85,86,87,84,84,82
1,1,2,1.0,Dan Lahav,Tel Aviv A,19,759,84.3,86,84,82,86,85,81,83,91,81
2,0,3,NaN,Kit Mercer,Durham A,23,753,83.7,83,85,82,85,80,82,83,85,88
3,0,3,NaN,Joshua Bailey,Durham A,23,753,83.7,84,85,82,83,82,83,83,85,86
4,0,5,NaN,Daniel Bramble,LSE A,21,749,83.2,79,85,83,81,82,85,86,83,85


In [41]:
# List the tabs we have for use in following code
list_of_tabs = [(wudc2017, "wudc2017"), (oxiv2016, "oxiv2016"), 
                (camiv2016, "camiv2016"),(eudc2016, "eudc2016")]

In [43]:
# Create a dictionary to store information
# defaultdict prevents KeyErrors
average_speaks_per_tournament = defaultdict(dict)

# Add average speaks and ESL status per speaker to our dictionary
for tournament in list_of_tabs:
    # Unpack tuple values
    tournament_name = tournament[1]
    tournament_tab = tournament[0]
    for row_index in range(len(tournament_tab)):
        row = tournament_tab.loc[row_index]
        speaker_name = row['Speaker']
        speaker_average = row['AVG']
        # Add tournament speaks average to speaker record
        speaker_record = average_speaks_per_tournament[speaker_name]
        speaker_record[tournament_name + "_avg"] = speaker_average
        # Add is_esl to speaker record
        speaker_record['is_esl_' + tournament_name] = row['is_esl']

In [44]:
# View dictionary 
# to check it's approx what we expect
average_speaks_per_tournament

defaultdict(dict,
            {'Nicholas Salmon': {'is_esl_wudc2017': 0,
              'wudc2017_avg': 80.599999999999994},
             'Lydia Day': {'is_esl_wudc2017': 0, 'wudc2017_avg': 79.0},
             'Shotaro Tsunoda': {'is_esl_wudc2017': 1,
              'wudc2017_avg': 71.099999999999994},
             'Manuel Kralj': {'eudc2016_avg': 75.799999999999997,
              'is_esl_eudc2016': 1},
             'Xinyue Wei': {'is_esl_wudc2017': 1,
              'wudc2017_avg': 77.599999999999994},
             'Jason Anderson': {'is_esl_wudc2017': 0,
              'wudc2017_avg': 71.900000000000006},
             "Rory O'Sullivan": {'eudc2016_avg': 81.900000000000006,
              'is_esl_eudc2016': 0},
             'Susanna Ulinski': {'camiv2016_avg': 74.0,
              'eudc2016_avg': 71.400000000000006,
              'is_esl_camiv2016': 1,
              'is_esl_eudc2016': 1},
             'Rayne Goddard': {'eudc2016_avg': 76.299999999999997,
              'is_esl_eudc2016': 0},

In [45]:
# Combine all esl indicators using OR

# this doesn't work! :(
# TODO: Fix this. For now am doing this in Google Sheets instead.
for speaker in average_speaks_per_tournament:
    # Should not hard code this but I'm lazy
    speaker['is_esl'] = speaker['is_esl_eudc2016'] \
                        or speaker['is_esl_wudc2017'] \
                        or speaker['is_esl_oxiv2016'] \
                        or speaker['is_esl_camiv2016']

TypeError: string indices must be integers

In [48]:
# Convert dictionary to a dataframe to export
average_speaks_per_tournament_df = \
    pd.DataFrame.from_dict(average_speaks_per_tournament, orient="index")

In [49]:
# View dataframe to check it's what we expect
average_speaks_per_tournament_df

,is_esl_wudc2017,wudc2017_avg,eudc2016_avg,is_esl_eudc2016,oxiv2016_avg,is_esl_oxiv2016,is_esl_camiv2016,camiv2016_avg
ABEL LAW,NaN,NaN,NaN,NaN,77.2,1.0,NaN,NaN
Aaron Luke,0.0,80.6,NaN,NaN,77.0,0.0,0.0,78.2
Aaron Schaffer-Neitz,0.0,77.2,NaN,NaN,75.4,0.0,0.0,76.6
Abhi Kulgod,0.0,76.2,NaN,NaN,NaN,NaN,NaN,NaN
Abhik Pant,1.0,70.1,NaN,NaN,NaN,NaN,NaN,NaN
Abhiroop Lohia,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.4
Abhisheka Dubey,0.0,78.2,NaN,NaN,NaN,NaN,NaN,NaN
Abigail Westberry,0.0,76.6,NaN,NaN,78.2,0.0,NaN,NaN
Abu Saleh Didar,1.0,77.1,NaN,NaN,NaN,NaN,NaN,NaN
Acca Kwai Ching Sum,1.0,76.1,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Attempt to combine ESL indicators again
# and fail again :(
for row_index in range(len(average_speaks_per_tournament_df)):
    row = average_speaks_per_tournament_df.iloc[row_index]
    row['is_esl'] = row['is_esl_wudc2017'] or row['is_esl_oxiv2016'] \
                    or row['is_esl_camiv2016'] or row['is_esl_eudc2016']

In [58]:
# View df again: combining ESL indicators is still not working!
average_speaks_per_tournament_df.head()

,is_esl_wudc2017,wudc2017_avg,eudc2016_avg,is_esl_eudc2016,oxiv2016_avg,is_esl_oxiv2016,is_esl_camiv2016,camiv2016_avg
ABEL LAW,NaN,NaN,NaN,NaN,77.2,1.0,NaN,NaN
Aaron Luke,0.0,80.6,NaN,NaN,77.0,0.0,0.0,78.2
Aaron Schaffer-Neitz,0.0,77.2,NaN,NaN,75.4,0.0,0.0,76.6
Abhi Kulgod,0.0,76.2,NaN,NaN,NaN,NaN,NaN,NaN
Abhik Pant,1.0,70.1,NaN,NaN,NaN,NaN,NaN,NaN
Abhiroop Lohia,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.4
Abhisheka Dubey,0.0,78.2,NaN,NaN,NaN,NaN,NaN,NaN
Abigail Westberry,0.0,76.6,NaN,NaN,78.2,0.0,NaN,NaN
Abu Saleh Didar,1.0,77.1,NaN,NaN,NaN,NaN,NaN,NaN
Acca Kwai Ching Sum,1.0,76.1,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# Convert the dataframe to a CSV 
# because I'm lazy and want to manipulate data in Google Sheets 
# and then run regs in Stata
average_speaks_per_tournament_df.to_csv(path_or_buf="avg_speaks_per_tournament_no_combined_esl_status.csv")

Deprecated code:
```
if speaker_record['is_esl'] is None or speaker_record['is_esl'] == 0:
            speaker_record['is_esl'] = row['is_esl']
```
Returns 
```
KeyError                                  Traceback (most recent call last)
<ipython-input-25-15a25a89b6b3> in <module>()
     10         speaker_record = average_speaks_per_tournament[speaker_name]
     11         speaker_record[tournament_name + "_avg"] = speaker_average
---> 12         if speaker_record['is_esl'] is None or speaker_record['is_esl'] == 0:
     13             speaker_record['is_esl'] = row['is_esl']

KeyError: 'is_esl'
```

## Suggestions
### Things to do suggestions
* Add dummy variables `top_quartile`, `bottom_quartile` or similar to see if ESL bias 'increases down the tab'.
    * May have issues with interpreting the coefficients
* Factoring in name or institutional recognition (though this probably doesn't affected ESL speakers exclusively or more than EPL speakers so it seems more appropriate for a separate study. And presumably effects apply across all tournaments so *shrugs*.)
* Region chairs are from
* Region speakers are from
* Identify where the drop in average speaks comes from (drop in speaker's highest speaks or lowest or across the board)
    * Hypothesis that bad speeches are judged more harshly at Oxbridge IVs (Roel Becker)
* Over-representation of high scoring speakers at Oxbridge (meh)
    * this will only matter in so far as it pulls up speaks in general since we're comparing an individual's speaks at WUDC vs Oxbridge IVs.
* Having judges who regularly judge certain ESL speakers and give them high speaks may make it easier for those judges to give 86s and 85s rather than 82s. (Berman)
    * Okay I don't see an easy way of correcting for this, also likely applies to both ESL and EPL?
    
### Method based suggestions
* Something two way ANOVA optionally with repeated measures
* Separate judges willing to make use of the 85-90 range (Joe Roussos).
    * If you look just at a small sample of teams (those that the top 15 speakers were in, say) then you're vulnerable to the confounding effect of encountering high-scoring judges more frequently at Cambridge, vs. at Worlds. 
    * and Effect of CA teams having different allocation strategies
        * In general WUDC seems to have allocated higher ranked judges closer to the middle of the tab than Cambridge's more manual system 
            * and proportion of ESL teams in top rooms may have been different (Berman)
                * This is valid only if for each ESL speaker, they are more likely to have been in a room with a high-speak-giving judge in Cam/Ox IV than not, and this effect is significantly different than for EPL speakers (whose speaks we used).
    * How to test for this / correct for it:
        1. Use Tabbie data across tournaments to track the kinds of scores judges give. Compare how judge A ranks a speaker, compared to the average speaks they get. In this way you can test my hypothesis and, if correct, identify the high scorers for step 2. 

        2. Correct for relative differences in contact with high scorers in your regression. Ideal comparison points are those with similar contact across all competitions tested.
        


The horrific Berman x Simpson thread


    
